In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import the training data
training_df = pd.read_csv('Raw_Data\sales_train_v2.csv')

# Import the items dataset
items_df = pd.read_csv('Raw_Data\items.csv')

# Import the item_categories.csv dataset with the item_category_id as the index
items_cat_df = pd.read_csv('Raw_Data\item_categories.csv', index_col = 'item_category_id')

# Import the shops.csv dataset with the shop_id as the index
shops_df = pd.read_csv('Raw_Data\shops.csv', index_col = 'shop_id')

# Import the test.csv dataset with the ID as the index
test = pd.read_csv('test.csv', index_col = 'ID')

# Import the sample_submission.csv dataset with the ID as the index
sample = pd.read_csv('Raw_Data\sample_submission.csv', index_col = 'ID')

In [3]:
training_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
# Define a function to convert the dates into a readable format and prepare it for datetime conversion
def fix_date_format(date):
    '''Changes format from DD.MM.YYYY to YYYY-MM-DD'''
    return (date[-4:] + '-' + date[3:5] + '-' + date[:2])

In [5]:
training_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
date              object
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [6]:
# Convert the date column to a datetime object.
training_df['date'] = training_df['date'].apply(fix_date_format)
datetime_conversion = pd.to_datetime(training_df['date'])
training_df['date'] = datetime_conversion

In [7]:
training_df.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [8]:
# Remove outliers in the item_price from our dataset.
training_df = training_df[(training_df['item_price'] != 307980.0) & (training_df['item_price'] >= 0)]

In [9]:
training_df.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935847e+06,2.935847e+06,2.935847e+06,2.935847e+06,2.935847e+06
mean,1.456992e+01,3.300174e+01,1.019723e+04,8.907489e+02,1.242641e+00
std,9.422989e+00,1.622697e+01,6.324298e+03,1.720490e+03,2.618835e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e-02,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,5.920000e+04,2.169000e+03


In [10]:
# Since we're excluding the item category and shop names, we only need two datasets: training_df and items_df

# The items dataset has 22170 unique item ids whereas the training dataset only has 21806 unique item ids
# We'll address this by using a right merge and filling in the necessary null values with the mode of each column
training_merge_df = training_df.merge(items_df, on='item_id', how='right')

# We'll also remove the item_name column here.
training_merge_df.drop('item_name', axis=1, inplace=True)

# Fill in the null values with the mode of each column
for col in training_merge_df:
    training_merge_df[col].fillna(training_merge_df[col].mode()[0], inplace=True) 

# Convert the date_block_num and shop_id from float to int type for better visualizations
training_merge_df[['date_block_num', 'shop_id']] = training_merge_df[['date_block_num', 'shop_id']].astype(int)

In [11]:
# For later analysis, we may want to use only the month or month + year for analysis (provides cylical data)
# We'll make new columns here for both the month and year.

training_merge_df['month'] = training_merge_df['date'].dt.month
training_merge_df['year'] = training_merge_df['date'].dt.year

In [11]:
# Write the working data into a csv file for later use
training_merge_df.to_csv('working_data.csv')